# Import modules and set core NRPy+ parameters

<a id='toc'></a>

## Table of Contents

1. [Step 1](#import): Import modules
1. [Step 2](#directory): Set C codes directory variables
1. [Step 3](#params): Set core NRPy+ parameters
1. [Step 4](#rk4): Generate Runge-Kutta timestepping code depending on Boundary Condition type
1. [Step 5](#timestep): Output find_timestep() function to C file
1. [Step 6](#pickle): Define function to pickle the C function dict in the code generation below
1. [Step 7](#id): Import initial data for the UIUC Black Hole
1. [Step 8](#bssn): Generate symbolic expressions for BSSN RHSs
1. [Step 9](#codegen): C codegen functions
    1. [Step 9.1](#bssn_codegen): Function to generate C code for BSSN RHS evaluation
    1. [Step 9.2](#ricci): Function to generate C code for computation of the Ricci tensor
    1. [Step 9.3](#sfrhs): Function to generate C code for Scalar Field RHS evaluation
    1. [Step 9.4](#ham): Function to generate C code for computing the Hamiltonian constraint violation
    1. [Step 9.5](#gammadet): Function to generate C code that enforces the constraint on the determinant of gamma
    1. [Step 9.6](#parallel): Run all the C codegen functions in parallel
    1. [Step 9.7](#cparams): Output C codes needed for declaring and setting Cparameters; also set 'free_parameters.h'
1. [Step 10](#bcs): Set up boundary conditions
    1. [Step 10.1](#parity): Write separate C function to apply inner parity conditions
1. [Step 11](#main): Main C code
1. [Step 12](#sbatch): Make sbatch files for Baltasar

<a id='import'></a>
## Import modules (Back to [top](#toc))

In [1]:
from outputC import lhrh, outCfunction, outC_function_dict
import finite_difference as fin
import NRPy_param_funcs as par
import grid as gri
import indexedexp as ixp
import reference_metric as rfm
import cmdline_helper as cmd
import sympy as sp
import shutil, os, sys, time
import pickle

<a id='directory'></a>
## Set C codes directory variables \[Back to [top](#toc)\]

In [2]:
# Create C code output directory
Ccodesdir = os.path.join("BSSN_LinearScalarFieldEvolution_Ccodes/")
# Remove C code output directory if it exists
shutil.rmtree(Ccodesdir, ignore_errors=True)
# Create a fresh directory
cmd.mkdir(Ccodesdir)

# Create executable output directory
outdir = os.path.join(Ccodesdir, "output/")
cmd.mkdir(outdir)

# Create out and err dirs (baltasar)
out = os.path.join(outdir, "out/")
err = os.path.join(outdir, "err/")
cmd.mkdir(out)
cmd.mkdir(err)

# Copy sbatch_example.sbatch
#sbatch = os.path.join("../", "archive/", "sbatch_example.sbatch")
#shutil.copy(sbatch, Ccodesdir)

<a id='params'></a>
## Set core NRPy+ parameters \[Back to [top](#toc)\]

In [3]:
# Set spatial dimension to 3
par.set_parval_from_str("grid::DIM", 3)
DIM = par.parval_from_str("grid::DIM")

# Enable/Disable SIMD-optimised code
SIMD_enable = True

# If SIMD is enabled, copy SIMD_intrinsics.h to the C codes directory
cmd.mkdir(os.path.join(Ccodesdir, "SIMD/"))
if SIMD_enable == True:
    shutil.copy(os.path.join("SIMD/", "SIMD_intrinsics.h"), os.path.join(Ccodesdir, "SIMD/"))
else:
    with open(os.path.join(Ccodesdir, "SIMD", "SIMD_intrinsics.h"), "w") as file:
        file.write("#define SIMD_IS_DISABLED\n")

# Set coordinate system and call the reference metric
CoordSystem = "SinhSpherical"
par.set_parval_from_str("reference_metric::CoordSystem", CoordSystem)
rfm.reference_metric()

# Set domain size and simulation time
domain_size = 1000
t_final = 2000

# Set number of outputs
N_outputs_factor = 2

# Set sinh width (for Sinh* coordinates)
sinh_width = 0.17

# Set params for ErfSpherical coordinates
erf_width = 2.8
erf_offset = 1.0

# Set params for GaussianSpherical coordinates
gauss_A = 0.9
gauss_B = 0.09
gauss_sig = 0.29

# Set boundary conditions type (QuadraticExtrapolation or Sommerfeld)
BC_Type = "Sommerfeld"

# Set finite differencing order
FD_order = 4
par.set_parval_from_str("finite_difference::FD_CENTDERIVS_ORDER", FD_order)

# Allow for the use of functions (instead of inline calculations) to compute finite differences
FD_functions_enable = True
par.set_parval_from_str("finite_difference::FD_functions_enable", FD_functions_enable)

# Set the core data type and the default CFL factor
REAL = "double"
default_CFL_FACTOR = 0.5

# Set mass and spin of the Black Hole
M = 1

# Toggle to evolve scalar field
scalar_field_evolve = False

# Toggle non-linear evolution
non_linear_evol = False

# Toggle psi_4 extraction
psi4_extraction = True
# l maximum for Spherical harmonic decomposition of psi4 (>= 2)
l_max = 4
# psi4 extraction radius
R_extraction = 30

# Set initial data function (Test, SchwarzschildScatter or QuasiBound or Zero)
ID_Func = "QuasiBound"
# Set initial data type for SchwarzschildScatter (Gaussian or Dipole)
ID_Type = "Dipole"

# Set frequency and axial quantum number for QuasiBound states
omega = 0.3929
m = 1

# Set axial symmetry
# par.set_parval_from_str("indexedexp::symmetry_axes","2")

<a id='rk4'></a>
## Generate Runge-Kutta timestepping code depending on Boundary Condition type \[Back to [top](#toc)\]

In [4]:
# Generate Runge-Kutta timestepping code
import MoLtimestepping.C_Code_Generation as MoL
from MoLtimestepping.RK_Butcher_Table_Dictionary import Butcher_dict

RK_method = "RK4"
RK_order = Butcher_dict[RK_method][1]
cmd.mkdir(os.path.join(Ccodesdir, "MoLtimestepping/"))

if BC_Type == "QuadraticExtrapolation":

    RHS_string = """
Ricci_eval(&rfmstruct, &params, RK_INPUT_GFS, auxevol_gfs);
rhs_eval(&rfmstruct, &params, auxevol_gfs, RK_INPUT_GFS, RK_OUTPUT_GFS);\n"""
    if scalar_field_evolve:
        RHS_string += "fields_rhs_eval(&rfmstruct, &params, RK_INPUT_GFS, RK_OUTPUT_GFS);\n"
    post_RHS_string = """
apply_bcs_curvilinear(&params, &bcstruct, NUM_EVOL_GFS, evol_gf_parity, RK_OUTPUT_GFS);
enforce_detgammabar_constraint(&rfmstruct, &params, RK_OUTPUT_GFS);\n"""

elif BC_Type == "Sommerfeld":

    RHS_string = """
Ricci_eval(&rfmstruct, &params, RK_INPUT_GFS, auxevol_gfs);
rhs_eval(&rfmstruct, &params, auxevol_gfs, RK_INPUT_GFS, RK_OUTPUT_GFS)\n;"""
    if scalar_field_evolve:
        RHS_string += "fields_rhs_eval(&rfmstruct, &params, RK_INPUT_GFS, RK_OUTPUT_GFS);\n"
    RHS_string += "apply_bcs_sommerfeld(&params, xx, &bcstruct, NUM_EVOL_GFS, evol_gf_parity, RK_INPUT_GFS, RK_OUTPUT_GFS);\n"
    post_RHS_string = """
apply_inner_parity_conditions(&params, &bcstruct, NUM_EVOL_GFS, evol_gf_parity, RK_OUTPUT_GFS);
enforce_detgammabar_constraint(&rfmstruct, &params, RK_OUTPUT_GFS);\n"""

MoL.MoL_C_Code_Generation(RK_method,
    RHS_string = RHS_string,
    post_RHS_string = post_RHS_string,
    outdir = os.path.join(Ccodesdir,"MoLtimestepping/"))

<a id='timestep'></a>
## Output find_timestep() function to C file \[Back to [top](#toc)\]

In [5]:
# Output the find_timestep() function to a C file
rfm.out_timestep_func_to_file(os.path.join(Ccodesdir, "find_timestep.h"))

<a id='pickle'></a>
## Define function to pickle the C function dict in the code generation below \[Back to [top](#toc)\]

In [6]:
# In the parallel C codegen below, the
def pickled_outC_function_dict(outC_function_dict):
    outstr = []
    outstr.append(pickle.dumps(len(outC_function_dict)))
    for Cfuncname, Cfunc in outC_function_dict.items():
        outstr.append(pickle.dumps(Cfuncname))
        outstr.append(pickle.dumps(Cfunc))
    return outstr

<a id='id'></a>
## Import initial data for the UIUC Black Hole \[Back to [top](#toc)\]

In [7]:
import BSSN.UIUCBlackHole as uiuc
import BSSN.ScalarFieldID as sfid

def UIUCBlackHoleAndFieldsID():
    print("Generating optimised C code for UIUC Black Hole AND Scalar Field initial data. May take a while, depending on CoordSystem.")
    start = time.time()

    uiuc.UIUCBlackHole(ComputeADMGlobalsOnly=True)    # Registers ID C function in dictionary, used below to output to file

    M          = uiuc.M
    chi        = uiuc.chi
    r          = uiuc.r
    th         = uiuc.th
    ph         = uiuc.ph
    gammaSphDD = uiuc.gammaSphDD
    KSphDD     = uiuc.KSphDD
    alphaSph   = uiuc.alphaSph
    betaSphU   = uiuc.betaSphU
    BSphU      = uiuc.BSphU

    Sph_r_th_ph = [r, th, ph]

    # Spin per unit mass
    a = M*chi

    # Boyer - Lindquist outer horizon
    rp = M + sp.sqrt(M**2 - a**2)
    # Boyer - Lindquist inner horizon
    rm = M - sp.sqrt(M**2 - a**2)

    # Boyer - Lindquist radius in terms of UIUC radius
    rBL = r*(1 + rp / (4*r))**2

    # Sigma = r_{BL}^2 + a^2 cos^2 theta
    SIG = rBL**2 + a**2*sp.cos(th)**2

    # Delta = r_{BL}^2 - 2Mr_{BL} + a^2
    DEL = rBL**2 - 2*M*rBL + a**2

    # A = (r_{BL}^2 + a^2)^2 - Delta a^2 sin^2 theta
    AA = (rBL**2 + a**2)**2 - DEL*a**2*sp.sin(th)**2

    # Lapse and shift for stationary spacetime
    stationary_alphaSph = sp.sqrt(DEL * SIG / AA)
    stationary_betaSph = -2 * M * a * rBL / AA

    import BSSN.ADM_Exact_Spherical_or_Cartesian_to_BSSNCurvilinear as AtoB
    Sph_r_th_ph = [r,th,ph]
    cf, hDD, lambdaU, aDD, trK, alpha, vetU, betU = \
        AtoB.Convert_Spherical_or_Cartesian_ADM_to_BSSN_curvilinear("Spherical", Sph_r_th_ph, gammaSphDD, KSphDD, alphaSph, betaSphU, BSphU)

    # Let's choose alpha = 1/psi**2 (psi = BSSN conformal factor) for these initial data,
    # where psi = exp(phi); chi = 1/psi**4; W = 1/psi**2
    if par.parval_from_str("EvolvedConformalFactor_cf") == "phi":
        alpha = sp.exp(-2*cf)
        psi = sp.exp(cf)
    elif par.parval_from_str("EvolvedConformalFactor_cf") == "chi":
        alpha = sp.sqrt(cf)
        psi = cf**(-sp.Rational(1,4))
    elif par.parval_from_str("EvolvedConformalFactor_cf") == "W":
        alpha = cf
        psi = cf**(-sp.Rational(1,2))
    else:
        print("Error EvolvedConformalFactor_cf type = \""+par.parval_from_str("EvolvedConformalFactor_cf")+"\" unknown.")
        sys.exit(1)

    if scalar_field_evolve:

        # Generate scalar field initial data expressions according to chosen function
        if ID_Func == "Test":
            sfid.ScalarFieldID()
        elif ID_Func == "SchwarzschildScatter":
            sfid.ScalarFieldID_Schwarzschild(psi, ID_Type)
        elif ID_Func == "QuasiBound":
            sfid.ScalarFieldID_QuasiBound(stationary_alphaSph, stationary_betaSph, Sph_r_th_ph)
        elif ID_Func == "Zero":
            sfid.ScalarFieldID_Zero()
        else:
            print(f"Initial data function {ID_Func} not supported!")
            exit(1)

        # Print the expressions to a file
        with open(os.path.join(Ccodesdir, "fields_initial_data.h"), "w") as file:
            file.write(outC_function_dict["fields_initial_data"])

    if non_linear_evol:
        if ID_Func == "SchwarzschildScatter":
            psi = sfid.psi
            if par.parval_from_str("EvolvedConformalFactor_cf") == "phi":
                cf = sp.log(psi)
            elif par.parval_from_str("EvolvedConformalFactor_cf") == "chi":
                cf = psi**(-4)
            elif par.parval_from_str("EvolvedConformalFactor_cf") == "W":
                cf = psi**(-2)

    import BSSN.BSSN_ID_function_string as bIDf
    # Generates initial_data() C function & stores to outC_function_dict["initial_data"]
    bIDf.BSSN_ID_function_string(cf, hDD, lambdaU, aDD, trK, alpha, vetU, betU)

    with open(os.path.join(Ccodesdir, "initial_data.h"), "w") as file:
        file.write(outC_function_dict["initial_data"])

    end = time.time()
    print(f"(BENCH) Finished UIUC BH and Scalar Field initial data codegen in {end - start} seconds.")
    return pickled_outC_function_dict(outC_function_dict)

In [8]:
#UIUCBlackHoleAndFieldsID()

<a id='bssn'></a>
## Generate symbolic expressions for BSSN RHSs \[Back to [top](#toc)\]

In [9]:
import BSSN.BSSN_RHSs as rhs
import BSSN.BSSN_gauge_RHSs as grhs
import BSSN.ScalarFieldRHSs as sfrhs

# Set the 1+log slicing condition
par.set_parval_from_str("BSSN.BSSN_gauge_RHSs::LapseEvolutionOption", "OnePlusLog")
# Set the *covariant*, second-order Gamma-driving shift condition
par.set_parval_from_str("BSSN.BSSN_gauge_RHSs::ShiftEvolutionOption", "GammaDriving2ndOrder_Covariant")

print("Generating symbolic expressions for BSSN RHSs AND Scalar Field RHSs...")
start = time.time()

# Enable rfm_precompute to eliminate transcendental functions from BSSN RHSs
cmd.mkdir(os.path.join(Ccodesdir, "rfm_files/"))
par.set_parval_from_str("reference_metric::enable_rfm_precompute", "True")
par.set_parval_from_str("reference_metric::rfm_precompute_Ccode_outdir", os.path.join(Ccodesdir, "rfm_files/"))

# Evaluate BSSN + BSSN gauge RHSs with rfm_precompute enabled
import BSSN.BSSN_quantities as Bq

# Setting LeaveRicciSymbolic to True leads to a significantly shorter RHS symbolic generation,
# at the small cost of a two-step MoL algorithm to compute RHSs in the C code
par.set_parval_from_str("BSSN.BSSN_quantities::LeaveRicciSymbolic", "True")

rhs.BSSN_RHSs()
grhs.BSSN_gauge_RHSs()

# Generate symbolic expressions for the scalar field RHSs
if scalar_field_evolve:
    sfrhs.ScalarFieldRHSs()

# Non-linear evolution
if non_linear_evol:

    # Generate expressions for the stress-energy tensor
    import BSSN.ScalarFieldT4UUmunu as sft
    sft.ScalarFieldSourceTerms()
    # T4UU = sft.T4UU
    sourceterm_trK_rhs = sft.sourceterm_trK_rhs
    sourceterm_Lambdabar_rhsU = sft.sourceterm_Lambdabar_rhsU
    sourceterm_lambda_rhsU = sft.sourceterm_lambda_rhsU
    sourceterm_a_rhsDD = sft.sourceterm_a_rhsDD

    # Generate BSSN source terms
    # import BSSN.BSSN_stress_energy_source_terms as Bsest
    # Bsest.BSSN_source_terms_for_BSSN_RHSs(T4UU)

    # Add BSSN source terms to BSSN RHS expressions
    rhs.trK_rhs += sourceterm_trK_rhs
    #print(sourceterm_trK_rhs)
    for i in range(DIM):
        # Needed for Gamma-driving shift RHSs:
        rhs.Lambdabar_rhsU[i] += sourceterm_Lambdabar_rhsU[i]
        # Needed for BSSN RHSs:
        rhs.lambda_rhsU[i]    += sourceterm_lambda_rhsU[i]
        for j in range(DIM):
            rhs.a_rhsDD[i][j] += sourceterm_a_rhsDD[i][j]

# Use betaU as the upwinding control vector
Bq.BSSN_basic_tensors()
betaU = Bq.betaU

# Generate symbolic expressions to enforce the constraint on the determinant of gammabar
import BSSN.Enforce_Detgammabar_Constraint as EGC
enforce_detg_constraint_symb_expressions = EGC.Enforce_Detgammabar_Constraint_symb_expressions()

# Compute the Ricci tensor
par.set_parval_from_str("BSSN.BSSN_quantities::LeaveRicciSymbolic", "False")
Bq.RicciBar__gammabarDD_dHatD__DGammaUDD__DGammaU()

# Register the Hamiltonian constraint as a grid function
H = gri.register_gridfunctions("AUX", "H")
# Define the Hamiltonian constraint and ouptut the optimised C code
import BSSN.BSSN_constraints as bssncon
bssncon.BSSN_constraints(add_T4UUmunu_source_terms=False)
if non_linear_evol:

    # cf = Bq.cf
    # if par.parval_from_str("EvolvedConformalFactor_cf") == "phi":
    #     psi = sp.exp(cf)
    # elif par.parval_from_str("EvolvedConformalFactor_cf") == "chi":
    #     psi = cf**(-sp.Rational(1,4))
    # elif par.parval_from_str("EvolvedConformalFactor_cf") == "W":
    #     psi = cf**(-sp.Rational(1,2))
    # else:
    #     print("Error EvolvedConformalFactor_cf type = \""+par.parval_from_str("EvolvedConformalFactor_cf")+"\" unknown.")
    #     sys.exit(1)

    # Bsest.BSSN_source_terms_for_BSSN_constraints(T4UU)
    # sourceterm_H = sft.sourceterm_H * psi**5
    # bssncon.H *= psi**5
    sourceterm_H = sft.sourceterm_H
    bssncon.H += sourceterm_H
    # bssncon.H = bssncon.H * 0 + sourceterm_H

# Restore all rfm hatted quantities to their close-form expressions by setting rfm_precompute to False
par.set_parval_from_str("reference_metric::enable_rfm_precompute", "False")
rfm.ref_metric__hatted_quantities()

# Print elapsed time
end = time.time()
print(f"(BENCH) Finished BSSN AND Scalar Field RHSs symbolic expression generation in {end - start} seconds.")

includes = None
if FD_functions_enable:
    includes = ["finite_difference_functions.h"]

Generating symbolic expressions for BSSN RHSs AND Scalar Field RHSs...
(BENCH) Finished BSSN AND Scalar Field RHSs symbolic expression generation in 7.209055423736572 seconds.


<a id='psi4_expressions'></a>
## Compute expressions for $\psi_4$ and output C code that imports all the necessary terms \[Back to [top](#toc)\]

In [10]:
if psi4_extraction:

    # Choose the QuasiKinnersley tetrad
    import BSSN.Psi4_tetrads as BP4t
    par.set_parval_from_str("BSSN.Psi4_tetrads::TetradChoice", "QuasiKinnersley")

    # Generate symbolic expressions for psi4
    import BSSN.Psi4 as BP4
    print("Generating symbolic expressions for psi4...")
    start = time.time()
    BP4.Psi4()
    end = time.time()
    print(f"(BENCH) Finished psi4 symbolic expressions in {end - start} seconds.")

    # Register aux gridfunctions for each term of the psi4 expressions for both real and imaginary parts
    psi4r_0pt = gri.register_gridfunctions("AUX","psi4r_0pt")
    psi4r_1pt = gri.register_gridfunctions("AUX","psi4r_1pt")
    psi4r_2pt = gri.register_gridfunctions("AUX","psi4r_2pt")
    psi4i_0pt = gri.register_gridfunctions("AUX","psi4i_0pt")
    psi4i_1pt = gri.register_gridfunctions("AUX","psi4i_1pt")
    psi4i_2pt = gri.register_gridfunctions("AUX","psi4i_2pt")

    # Output C code to compute each individual term of psi4
    desc="""Since it's so expensive to compute, instead of evaluating
psi_4 at all interior points, this functions evaluates it on a
point-by-point basis."""
    name="psi4"
    outCfunction(
        outfile  = os.path.join(Ccodesdir,name+".h"), desc=desc, name=name,
        params   = """const paramstruct *restrict params,
                      const int i0,const int i1,const int i2,
                      REAL *restrict xx[3], const REAL *restrict in_gfs, REAL *restrict aux_gfs""",
        body     = """
    const int idx = IDX3S(i0,i1,i2);
    const REAL xx0 = xx[0][i0];const REAL xx1 = xx[1][i1];const REAL xx2 = xx[2][i2];
// Real part of psi_4, divided into 3 terms
    {
#include "Psi4re_pt0_lowlevel.h"
    }
    {
#include "Psi4re_pt1_lowlevel.h"
    }
    {
#include "Psi4re_pt2_lowlevel.h"
    }
// Imaginary part of psi_4, divided into 3 terms
    {
#include "Psi4im_pt0_lowlevel.h"
    }
    {
#include "Psi4im_pt1_lowlevel.h"
    }
    {
#include "Psi4im_pt2_lowlevel.h"
    }""")

Generating symbolic expressions for psi4...
(BENCH) Finished psi4 symbolic expressions in 27.036287307739258 seconds.
Output C function psi4() to file BSSN_LinearScalarFieldEvolution_Ccodes/psi4.h


<a id='spherical_harmonics'></a>
## Output C code to compute spin-weight -2 spherical harmonics ${}^{-2}Y_{\ell,m}$ up to $\ell = \ell_{\text{max}}$ \[Back to [top](#toc)\]

In [11]:
if psi4_extraction:
    import SpinWeight_minus2_SphHarmonics.SpinWeight_minus2_SphHarmonics as swm2
    cmd.mkdir(os.path.join(Ccodesdir, "SpinWeight_minus2_SphHarmonics"))
    swm2.SpinWeight_minus2_SphHarmonics(maximum_l=l_max,
                                       filename=os.path.join(Ccodesdir, "SpinWeight_minus2_SphHarmonics", "SpinWeight_minus2_SphHarmonics.h"))

<a id='codegen'></a>
## C codegen functions \[Back to [top](#toc)\]

<a id='bssn_codegen'></a>
### Function to generate C code for BSSN RHS evaluation \[Back to [top](#toc)\]

In [12]:
# Function to output C code for computing the BSSN right-hand sides
def BSSN_RHSs():
    print(f"Generating C code for BSSN RHSs in {par.parval_from_str('reference_metric::CoordSystem')} coordinates.")
    start = time.time()

    # Construct the left-hand sides and right-hand side expressions for all BSSN RHSs
    lhs_names = [    "alpha",       "cf",       "trK"]
    rhs_exprs = [grhs.alpha_rhs, rhs.cf_rhs, rhs.trK_rhs]
    for i in range(3):
        lhs_names.append(   f"betU{i}")
        rhs_exprs.append(grhs.bet_rhsU[i])
        lhs_names.append(  f"lambdaU{i}")
        rhs_exprs.append(rhs.lambda_rhsU[i])
        lhs_names.append(   f"vetU{i}")
        rhs_exprs.append(grhs.vet_rhsU[i])
        for j in range(i, 3):
            lhs_names.append(f"aDD{i}{j}")
            rhs_exprs.append(rhs.a_rhsDD[i][j])
            lhs_names.append(f"hDD{i}{j}")
            rhs_exprs.append(rhs.h_rhsDD[i][j])

    # Sort the lhss list alphabetically, and the rhss to match
    lhs_names, rhs_exprs = [list(x) for x in zip(*sorted(zip(lhs_names, rhs_exprs), key=lambda pair: pair[0]))]

    # Declare the list of lhrh's
    BSSN_evol_rhss = []
    for var in range(len(lhs_names)):
        BSSN_evol_rhss.append(lhrh(lhs=gri.gfaccess("rhs_gfs", lhs_names[var]), rhs=rhs_exprs[var]))

    # Set up the C function for the BSSN RHSs
    # Set up the outputC and loop parameters for BSSN_RHSs C function
    outC_params = "outCverbose=False"
    loopoptions = "InteriorPoints,Enable_rfm_precompute"
    if SIMD_enable == True:
        loopoptions += ",EnableSIMD,OMP_custom_pragma='#pragma omp parallel for collapse(3)'"
        outC_params += ",SIMD_enable=True"
    desc = "Evaluate the BSSN RHSs"
    name = "rhs_eval"
    outCfunction(
        outfile = os.path.join(Ccodesdir, name + ".h"), includes=includes, desc=desc, name=name,
        params = """rfm_struct *restrict rfmstruct, const paramstruct *restrict params,
                    const REAL *restrict auxevol_gfs, const REAL *restrict in_gfs, REAL *restrict rhs_gfs""",
        body = fin.FD_outputC("returnstring", BSSN_evol_rhss, params=outC_params, upwindcontrolvec=betaU).replace("IDX4", "IDX4S"),
        loopopts=loopoptions
    )

    # Print elapsed time
    end = time.time()
    print(f"(BENCH) Finished BSSN_RHS C codegen in {end - start} seconds.")
    return pickled_outC_function_dict(outC_function_dict)

<a id='ricci'></a>
### Function to generate C code for computation of the Ricci tensor \[Back to [top](#toc)\]

In [13]:
# Function to output C code for computing the Ricci tensor
def Ricci():
    print(f"Generating C code for Ricci tensor in {par.parval_from_str('reference_metric::CoordSystem')} coordinates.")
    start = time.time()

    # Set up the C function for the Ricci tensor
    # Set outputC and loop parameters for Ricci tensor function
    outC_params = "outCverbose=False"
    loopoptions = "InteriorPoints,Enable_rfm_precompute"
    if SIMD_enable == True:
        loopoptions += ",EnableSIMD,OMP_custom_pragma='#pragma omp parallel for collapse(3)'"
        outC_params += ",SIMD_enable=True"
    desc = "Evaluate the Ricci tensor"
    name = "Ricci_eval"
    outCfunction(
        outfile = os.path.join(Ccodesdir, name+".h"), includes=includes, desc=desc, name=name,
        params = """rfm_struct *restrict rfmstruct, const paramstruct *restrict params,
                    const REAL *restrict in_gfs, REAL *restrict auxevol_gfs""",
        body = fin.FD_outputC("returnstring",
                              [lhrh(lhs=gri.gfaccess("auxevol_gfs","RbarDD00"),rhs=Bq.RbarDD[0][0]),
                               lhrh(lhs=gri.gfaccess("auxevol_gfs","RbarDD01"),rhs=Bq.RbarDD[0][1]),
                               lhrh(lhs=gri.gfaccess("auxevol_gfs","RbarDD02"),rhs=Bq.RbarDD[0][2]),
                               lhrh(lhs=gri.gfaccess("auxevol_gfs","RbarDD11"),rhs=Bq.RbarDD[1][1]),
                               lhrh(lhs=gri.gfaccess("auxevol_gfs","RbarDD12"),rhs=Bq.RbarDD[1][2]),
                               lhrh(lhs=gri.gfaccess("auxevol_gfs","RbarDD22"),rhs=Bq.RbarDD[2][2])],
                               params=outC_params).replace("IDX4", "IDX4S"),
        loopopts=loopoptions
    )

    # Print elapsed time
    end = time.time()
    print(f"(BENCH) Finished Ricci C codegen in {end - start} seconds.")
    return pickled_outC_function_dict(outC_function_dict)

<a id='sfrhs'></a>
### Function to generate C code for Scalar Field RHS evaluation \[Back to [top](#toc)\]

In [14]:
def ScalarField_RHSs():

    # Start timer
    print(f"Generating C code for Scalar Field RHSs in {par.parval_from_str('reference_metric::CoordSystem')} coordinates.")
    start = time.time()

    # Output to C code

    # Construct the LHSs and RHS expressions for the fields
    lhs_names = [      "Phi",         "Pi"]
    rhs_exprs = [ sfrhs.Phi_rhs, sfrhs.Pi_rhs]

    # Sort the lhss list alphabetically, and rhss to match.
    #   This ensures the RHSs are evaluated in the same order
    #   they're allocated in memory:
    # lhs_names, rhs_exprs = [list(x) for x in zip(*sorted(zip(lhs_names, rhs_exprs), key=lambda pair: pair[0]))]

    # Declare the list of lhrh's
    fields_rhss = []
    for var in range(len(lhs_names)):
        fields_rhss.append(lhrh(lhs=gri.gfaccess("rhs_gfs", lhs_names[var]), rhs=rhs_exprs[var]))

    # Set up the C function for evaluating the RHSs for the fields
    outC_params = "outCverbose=False"
    loopoptions = "InteriorPoints,Enable_rfm_precompute"
    if SIMD_enable == True:
        loopoptions += ",EnableSIMD,OMP_custom_pragma='#pragma omp parallel for collapse(3)'"
        outC_params += ",SIMD_enable=True"
    desc = "Evaluate the scalar fields RHSs"
    name = "fields_rhs_eval"
    outCfunction(
        outfile = os.path.join(Ccodesdir, name + ".h"),
        desc = desc,
        name = name,
        params = "rfm_struct *restrict rfmstruct, const paramstruct *restrict params, const REAL *restrict in_gfs, REAL *restrict rhs_gfs",
        body = fin.FD_outputC("returnstring", fields_rhss, params=outC_params, upwindcontrolvec=betaU).replace("IDX4", "IDX4S"),
        loopopts = loopoptions
    )

    # End timer
    end = time.time()
    print(f'(BENCH) Finished Scalar Field RHSs C codegen in {end - start} seconds.')
    return pickled_outC_function_dict(outC_function_dict)

<a id='ham'></a>
### Function to generate C code for computing the Hamiltonian constraint violation \[Back to [top](#toc)\]

In [15]:
# Function to output C code for computing the Hamiltonian constraint
def Hamiltonian():
    print("Generating optimised C code for the Hamiltonian constraint. May take a while, depending on CoordSystem.")
    start = time.time()

    # Set up the C function for the Hamiltonian RHS
    desc = "Evaluate the Hamiltonian constraint"
    name = "Hamiltonian_constraint"
    outCfunction(
        outfile = os.path.join(Ccodesdir, name + ".h"), includes=includes, desc=desc, name=name,
        params = """rfm_struct *restrict rfmstruct, const paramstruct *restrict params,
                    REAL *restrict in_gfs, REAL *restrict aux_gfs""",
        body = fin.FD_outputC("returnstring", lhrh(lhs=gri.gfaccess("aux_gfs", "H"), rhs=bssncon.H),
                               params="outCverbose=False").replace("IDX4", "IDX4S"),
        loopopts="InteriorPoints,Enable_rfm_precompute,OMP_custom_pragma='#pragma omp parallel for collapse(3)'"
    )

    # Print elapsed time
    end = time.time()
    print(f"(BENCH) Finished Hamiltonian C codegen in {end - start} seconds.")
    return pickled_outC_function_dict(outC_function_dict)

<a id='gammadet'></a>
### Function to generate C code that enforces the constraint on the determinant of gamma \[Back to [top](#toc)\]

In [16]:
# Function to output C code for enforcing the constraint on the determinant of gammabar
def gammadet():
    print("Generating optimised C code for gamma constraint. May take a while, depending on CoordSystem.")
    start = time.time()

    # Set up the C function for the det(gammahat) = det(gammabar)
    EGC.output_Enforce_Detgammabar_Constraint_Ccode(Ccodesdir, exprs=enforce_detg_constraint_symb_expressions)

    # Print elapsed time
    end = time.time()
    print(f"(BENCH) Finished gamma constraint C codegen in {end - start} seconds.")
    return pickled_outC_function_dict(outC_function_dict)

<a id='psi4_ccode'></a>
### Function to generate C code for the each real or imaginary term of $\psi_4$ \[Back to [top](#toc)\]

In [17]:
def Psi4re(part):
    print("Generating C code for psi4_re_pt"+str(part)+" in "+par.parval_from_str("reference_metric::CoordSystem")+" coordinates.")
    start = time.time()
    Psi4re_pt = fin.FD_outputC("returnstring",
                   [lhrh(lhs=gri.gfaccess("aux_gfs","psi4r_"+str(part)+"pt"),rhs=BP4.psi4_re_pt[part])],
                   params="outCverbose=False,CSE_sorting=none") # Generating the CSE for psi4 is the slowest
                                                                # operation in this notebook, and much of the CSE
                                                                # time is spent sorting CSE expressions. Disabling
                                                                # this sorting makes the C codegen 3-4x faster,
                                                                # but the tradeoff is that every time this is
                                                                # run, the CSE patterns will be different
                                                                # (though they should result in mathematically
                                                                # *identical* expressions). You can expect
                                                                # roundoff-level differences as a result.
    with open(os.path.join(Ccodesdir,"Psi4re_pt"+str(part)+"_lowlevel.h"), "w") as file:
        file.write(Psi4re_pt.replace("IDX4","IDX4S"))
    end = time.time()
    print("(BENCH) Finished generating psi4_re_pt"+str(part)+" in "+str(end-start)+" seconds.")
    return pickled_outC_function_dict(outC_function_dict)

def Psi4im(part):
    print("Generating C code for psi4_im_pt"+str(part)+" in "+par.parval_from_str("reference_metric::CoordSystem")+" coordinates.")
    start = time.time()
    Psi4im_pt = fin.FD_outputC("returnstring",
                   [lhrh(lhs=gri.gfaccess("aux_gfs","psi4i_"+str(part)+"pt"),rhs=BP4.psi4_im_pt[part])],
                   params="outCverbose=False,CSE_sorting=none") # Generating the CSE for psi4 is the slowest
                                                                # operation in this notebook, and much of the CSE
                                                                # time is spent sorting CSE expressions. Disabling
                                                                # this sorting makes the C codegen 3-4x faster,
                                                                # but the tradeoff is that every time this is
                                                                # run, the CSE patterns will be different
                                                                # (though they should result in mathematically
                                                                # *identical* expressions). You can expect
                                                                # roundoff-level differences as a result.
    with open(os.path.join(Ccodesdir,"Psi4im_pt"+str(part)+"_lowlevel.h"), "w") as file:
        file.write(Psi4im_pt.replace("IDX4","IDX4S"))
    end = time.time()
    print("(BENCH) Finished generating psi4_im_pt"+str(part)+" in "+str(end-start)+" seconds.")
    return pickled_outC_function_dict(outC_function_dict)

<a id='parallel'></a>
### Run all the C codegen functions in parallel \[Back to [top](#toc)\]

In [18]:
# Generate C code kernels for BSSN expressions, in parallel if possible

# Create a list of functions we wish to evaluate in parallel (if possible)
if psi4_extraction:
    funcs = [Psi4re, Psi4re, Psi4re, Psi4im, Psi4im, Psi4im, UIUCBlackHoleAndFieldsID, BSSN_RHSs, Ricci, Hamiltonian, gammadet]
else:
    funcs = [UIUCBlackHoleAndFieldsID, BSSN_RHSs, Ricci, Hamiltonian, gammadet]

if scalar_field_evolve:
    funcs.append(ScalarField_RHSs)
    # funcs.append(ScalarField_ID)
# pickled_outC_func_dict stores outC_function_dict from all the subprocesses in the parallel codegen
pickled_outC_func_dict = []

parallelise = True

try:
    if os.name == 'nt':
        # It's a mess to get working in Windows, so we don't bother. :/
        #  https://medium.com/@grvsinghal/speed-up-your-python-code-using-multiprocessing-on-windows-and-jupyter-or-ipython-2714b49d6fac
        raise Exception("Parallel codegen currently not available in certain environments, e.g., Windows")

    if not parallelise:
        raise Exception("Parallel codegen turned off. Proceeding to serival evaluation.")

    # Import the multiprocessing module.
    import multiprocessing as mp

    # Define master functions for parallelisation.
    def master_func(arg):
        if psi4_extraction:
            if arg < 3: # Call Psi4re(arg)
                return funcs[arg](arg)
            elif arg < 6: # Call Psi4im(arg-3)
                return funcs[arg](arg-3)
            else: # All non-Psi4 functions:
                return funcs[arg]()
        else:
            return funcs[arg]()
    # Evaluate list of functions in parallel if possible
    # otherwise fallback to serial evaluation
    pool = mp.Pool(mp.cpu_count())
    pickled_outC_func_dict.append(pool.map(master_func,range(len(funcs))))
    pool.terminate()
    pool.join()
except:
    # Fallback to serial evaluation if parallel not possible
    for func in funcs:
        func()
    pickled_outC_func_dict = []

# Output functions for computing all finite-difference stencils
if FD_functions_enable and len(pickled_outC_func_dict)>0:
    # First unpickle pickled_outC_func_dict
    outCfunc_dict = {}
    for WhichFunc in pickled_outC_func_dict[0]:
        i = 0
        num_elements = pickle.loads(WhichFunc[i]); i += 1
        for lst in range(num_elements):
            funcname = pickle.loads(WhichFunc[i])
            funcbody = pickle.loads(WhichFunc[i + 1]); i += 2
            outCfunc_dict[funcname] = funcbody
    # Then store the unpickled outCfunc_dict to outputC's outC_function_dict
    for key, item in outCfunc_dict.items():
        outC_function_dict[key] = item

if FD_functions_enable:
    # Finally generate finite_difference_functions.h
    fin.output_finite_difference_functions_h(path=Ccodesdir)

Generating C code for psi4_re_pt1 in SinhSpherical coordinates.
Generating C code for psi4_re_pt0 in SinhSpherical coordinates.
Generating C code for psi4_im_pt0 in SinhSpherical coordinates.
Generating C code for psi4_re_pt2 in SinhSpherical coordinates.
Generating C code for psi4_im_pt1 in SinhSpherical coordinates.
Generating optimised C code for the Hamiltonian constraint. May take a while, depending on CoordSystem.
Generating C code for BSSN RHSs in SinhSpherical coordinates.
Generating C code for psi4_im_pt2 in SinhSpherical coordinates.
Generating C code for Ricci tensor in SinhSpherical coordinates.
Generating optimised C code for UIUC Black Hole AND Scalar Field initial data. May take a while, depending on CoordSystem.
Generating optimised C code for gamma constraint. May take a while, depending on CoordSystem.
Generating C code for Scalar Field RHSs in SinhSpherical coordinates.
Output C function enforce_detgammabar_constraint() to file BSSN_LinearScalarFieldEvolution_Ccodes/

<a id='cparams'></a>
### Output C codes needed for declaring and setting Cparameters; also set `free_parameters.h` \[Back to [top](#toc)\]

In [19]:
# Output C codes needed for declaring and setting Cparameters; also set free_parameters.h

# Generate declare_Cparameters_struct.h, set_Cparameters_default.h and set_Cparameters[-SIMD].h
par.generate_Cparameters_Ccodes(os.path.join(Ccodesdir))

# Append to $Ccodesdir/free_parameters.h reference metric parameters based on generic
# domain_size,sinh_width,sinhv2_const_dr,SymTP_bScale, parameters set above.
rfm.out_default_free_parameters_for_rfm(os.path.join(Ccodesdir,"free_parameters.h"), domain_size, sinh_width)

# Set free_parameters.h
with open(os.path.join(Ccodesdir, "free_parameters.h"), "a") as file:

    file.write(f"""
// Set final simulation time
REAL t_final = {t_final};

// Set number of outputs
int N_outputs_factor = {N_outputs_factor};

// Set variable for the coordinate system
char coord_system[] = "{CoordSystem}";

// Set Black Hole mass
params.M = {M};""")

    if scalar_field_evolve:
        file.write(f"""
// Set parameters for pseudo-bound state
params.omega = {omega};
params.m = {m};
    """)

    if psi4_extraction:
        file.write(f"""
const REAL R_EXTRACT = {R_extraction};""")

    if CoordSystem == "ErfSpherical":
        file.write(f"""
params.ERFW = {erf_width};
params.OFFSET = {erf_offset};""")
    elif CoordSystem == "GaussianSpherical":
        file.write(f"""
params.GAUSSIAN_AMPL = {gauss_A};
params.GAUSSIAN_OFFSET = {gauss_B};
params.GAUSSIAN_SIG = {gauss_sig};""")

# Set coord_defines.h
with open(os.path.join(Ccodesdir, "coord_defines.h"), "w") as file:

    if CoordSystem == "Spherical":
        file.write("""
#define SPHERICAL 1.0""")
    elif CoordSystem == "SinhSpherical":
        file.write("""
#define SINHSPHERICAL 1.0""")
    elif CoordSystem == "ErfSpherical":
        file.write("""
#define ERFSPHERICAL 1.0""")
    elif CoordSystem == "GaussianSpherical":
        file.write(f"""
#define GAUSSIANSPHERICAL 1.0""")

    if psi4_extraction:
        file.write(f"""
#define PSI4 1.0
#define L_MAX {l_max}""")

# Generate set_Nxx_dxx_invdx_params__and__xx.h
rfm.set_Nxx_dxx_invdx_params__and__xx_h(Ccodesdir)

# Generate xxCart.h to map from xx to cartesian coordinates
rfm.xxCart_h("xxCart", "./set_Cparameters.h", os.path.join(Ccodesdir, "xxCart.h"))

# Generate declare_Cparameters_struct.h, set_Cparameters_default.h and set_Cparameters[-SIMD].h
par.generate_Cparameters_Ccodes(os.path.join(Ccodesdir))

<a id='psi4_drive'></a>
### Output C code to drive the extraction of $\psi_4$ and its decomposition into spin-weight -2 spherical harmonics (Back to [top](#toc))

In [20]:
%%writefile $Ccodesdir/psi4_driver.h

void psi4_driver(const paramstruct *restrict params, const REAL current_time,
                 const int R_extraction_idx, REAL *restrict xx[3],
                 const REAL *restrict y_n_gfs, REAL *restrict diagnostic_output_gfs) {
    #include "set_Cparameters.h"

    // Set the extraction radius R_extraction based on the radial index R_extraction_idx
    REAL R_extraction;
    {
        REAL xCart[3];
        xxCart(params, xx, R_extraction_idx, 1, 1, xCart);
        R_extraction = sqrt(xCart[0] * xCart[0] + xCart[1] * xCart[1] + xCart[2] * xCart[2]);
    }

    // Compute psi_4 at the extraction radius and stor it to an array
    const int sizeof_2Darray = sizeof(REAL) * (Nxx_plus_2NGHOSTS1 - 2 * NGHOSTS) * (Nxx_plus_2NGHOSTS2 - 2 * NGHOSTS);
    REAL *restrict psi4r_at_R_extraction = (REAL * restrict) malloc(sizeof_2Darray);
    REAL *restrict psi4i_at_R_extraction = (REAL *  restrict) malloc(sizeof_2Darray);

    // Store also theta, sin(theta) and phi to arrays
    REAL *restrict th_array = (REAL * restrict) malloc(sizeof(REAL) * (Nxx_plus_2NGHOSTS1 - 2 * NGHOSTS));
    REAL *restrict sinth_array = (REAL * restrict) malloc(sizeof(REAL) * (Nxx_plus_2NGHOSTS1 - 2 * NGHOSTS));
    REAL *restrict ph_array = (REAL * restrict) malloc(sizeof(REAL) * (Nxx_plus_2NGHOSTS2 - 2 * NGHOSTS));

    // Loop over theta and phi and fill the arrays
    const int i0 = R_extraction_idx;
    #pragma omp parallel for
    for (int i1 = NGHOSTS; i1 < Nxx_plus_2NGHOSTS1 - NGHOSTS; i1++) {
        th_array[i1 - NGHOSTS] = xx[1][i1];
        sinth_array[i1 - NGHOSTS] = sin(xx[1][i1]);
        for (int i2 = NGHOSTS; i2 < Nxx_plus_2NGHOSTS2 - NGHOSTS; i2++) {
            ph_array[i2 - NGHOSTS] = xx[2][i2];

            // Compute real and imaginary parts of psi_4 and output to diagnostic_output_gfs
            psi4(params, i0, i1, i2, xx, y_n_gfs, diagnostic_output_gfs);
            const int idx3d = IDX3S(i0, i1, i2);
            const REAL psi4r = (+ diagnostic_output_gfs[IDX4ptS(PSI4R_0PTGF, idx3d)]
                                + diagnostic_output_gfs[IDX4ptS(PSI4R_1PTGF, idx3d)]
                                + diagnostic_output_gfs[IDX4ptS(PSI4R_2PTGF, idx3d)]);
            const REAL psi4i = (+ diagnostic_output_gfs[IDX4ptS(PSI4I_0PTGF, idx3d)]
                                + diagnostic_output_gfs[IDX4ptS(PSI4I_1PTGF, idx3d)]
                                + diagnostic_output_gfs[IDX4ptS(PSI4I_2PTGF, idx3d)]);

            // Store the result to a "2D" array (actually 1D array with 2D storage)
            const int idx2d = (i1 - NGHOSTS) * (Nxx_plus_2NGHOSTS2 - 2 * NGHOSTS) + (i2 - NGHOSTS);
            psi4r_at_R_extraction[idx2d] = psi4r;
            psi4i_at_R_extraction[idx2d] = psi4i;
        }
    }

    // Perform integrations across all l,m modes from l=2 up to and including L_MAX (global variable)
    psi4_integration(params, current_time, R_extraction, th_array, sinth_array, ph_array, psi4r_at_R_extraction, psi4i_at_R_extraction);

    // Free allocated memory
    free(psi4r_at_R_extraction);
    free(psi4i_at_R_extraction);
    free(th_array);
    free(sinth_array);
    free(ph_array);
}


Writing BSSN_LinearScalarFieldEvolution_Ccodes//psi4_driver.h


In [21]:
%%writefile $Ccodesdir/psi4_integration.h

void psi4_integration(const paramstruct *restrict params, const REAL current_time,
                      const REAL R_extraction, const REAL *restrict th_array,
                      const REAL *restrict sinth_array, const REAL *restrict ph_array,
                      const REAL *restrict psi4r_at_R_extraction, const REAL *restrict psi4i_at_R_extraction) {
    #include "set_Cparameters.h"

    // Loop over all values of 2 <= l <= L_MAX
    for (int l = 2; l <= L_MAX; l++) {
        // Loop over -l <= m <= l
        for (int m = -l; m <= l; m++) {
            // Parallelize integration loop
            REAL psi4r_l_m = 0.0;
            REAL psi4i_l_m = 0.0;
            #pragma omp parallel for reduction(+: psi4r_l_m, psi4i_l_m)
            for (int i1 = 0; i1 <= Nxx_plus_2NGHOSTS1 - 2 * NGHOSTS; i1++) {

                const REAL th = th_array[i1];
                const REAL sinth = sinth_array[i1];

                for (int i2 = 0; i2 < Nxx_plus_2NGHOSTS2 - 2 * NGHOSTS; i2++) {

                    const REAL ph = ph_array[i2];

                    // Construct integrand for psi4 spin-weight s=-2 spherical harmonic
                    REAL ReY_sm2_l_m, ImY_sm2_l_m;
                    SpinWeight_minus2_SphHarmonics(l, m, th, ph, &ReY_sm2_l_m, &ImY_sm2_l_m);

                    const int idx2d = i1 * (Nxx_plus_2NGHOSTS2 - 2 * NGHOSTS) + i2;
                    const REAL a = psi4r_at_R_extraction[idx2d];
                    const REAL b = psi4i_at_R_extraction[idx2d];
                    const REAL c = ReY_sm2_l_m;
                    const REAL d = ImY_sm2_l_m;
                    psi4r_l_m += (a * c + b * d) * dxx2 * sinth * dxx1;
                    psi4i_l_m += (b * c - a * d) * dxx2 * sinth * dxx1;
                }
            }

            // Output the result of the integration to file
            char filename[100];
            if (m >= 0) sprintf(filename, "psi4_l%d_m+%d_%d_r%.2f.txt", l, m, Nxx0, (double) R_extraction);
            else sprintf(filename, "psi4_l%d_m%d_%d_r%.2f.txt", l, m, Nxx0, (double)R_extraction);
            FILE *outpsi4_l_m;
            if (current_time == 0) outpsi4_l_m = fopen(filename, "w");
            else outpsi4_l_m = fopen(filename, "a");
            fprintf(outpsi4_l_m, "%e %.15e %.15e\n", (double) current_time, (double) psi4r_l_m, (double) psi4i_l_m);
            fclose(outpsi4_l_m);
        }
    }
}


Writing BSSN_LinearScalarFieldEvolution_Ccodes//psi4_integration.h


<a id='bcs'></a>
## Set up boundary conditions \[Back to [top](#toc)\]

In [22]:
import CurviBoundaryConditions.CurviBoundaryConditions as cbcs
cbcs.Set_up_CurviBoundaryConditions(os.path.join(Ccodesdir,"boundary_conditions/"),Cparamspath=os.path.join("../"), BoundaryCondition=BC_Type)

if BC_Type == "Sommerfeld":
    bcs = cbcs.sommerfeld_bc(vars_radial_falloff_power_default=3., vars_speed_default=1., vars_at_inf_default=0.)

    # Set value for the lapse at infinity to 1
    bcs.vars_at_infinity["ALPHAGF"] = 1.0
    bcs.vars_at_infinity["CFGF"] = 1.0

    # We could change values of GF speed or values at infinity, but the defaults work for now
    # Write values to parameter file
    bcs.write_sommerfeld_files(Ccodesdir, fd_order=2)

Wrote to file "BSSN_LinearScalarFieldEvolution_Ccodes/boundary_conditions/parity_conditions_symbolic_dot_products.h"
Evolved parity: ( Phi:0, Pi:0, aDD00:4, aDD01:5, aDD02:6, aDD11:7, aDD12:8,
    aDD22:9, alpha:0, betU0:1, betU1:2, betU2:3, cf:0, hDD00:4, hDD01:5,
    hDD02:6, hDD11:7, hDD12:8, hDD22:9, lambdaU0:1, lambdaU1:2, lambdaU2:3,
    trK:0, vetU0:1, vetU1:2, vetU2:3 )
Auxiliary parity: ( H:0, psi4i_0pt:0, psi4i_1pt:0, psi4i_2pt:0,
    psi4r_0pt:0, psi4r_1pt:0, psi4r_2pt:0 )
AuxEvol parity: ( RbarDD00:4, RbarDD01:5, RbarDD02:6, RbarDD11:7,
    RbarDD12:8, RbarDD22:9 )
Wrote to file "BSSN_LinearScalarFieldEvolution_Ccodes/boundary_conditions/EigenCoord_Cart_to_xx.h"


<a id='parity'></a>
### Write separate C function to apply inner parity conditions \[Back to [top](#toc)\]

In [23]:
%%writefile $Ccodesdir/boundary_conditions/apply_inner_parity_conditions.h

void apply_inner_parity_conditions(const paramstruct *restrict params, const bc_struct *restrict bcstruct, const int NUM_GFS, const int8_t *restrict gfs_parity, REAL *restrict gfs) {

#include "../set_Cparameters.h"

#pragma omp parallel for
    for (int which_gf = 0; which_gf < NUM_GFS; which_gf++) {
        for (int which_gz = 0; which_gz < NGHOSTS; which_gz++) {
            for (int pt = 0; pt < bcstruct->num_ib_gz_pts[which_gz]; pt++)
            {
                const int i0dest = bcstruct->inner[which_gz][pt].inner_bc_dest_pt.i0;
                const int i1dest = bcstruct->inner[which_gz][pt].inner_bc_dest_pt.i1;
                const int i2dest = bcstruct->inner[which_gz][pt].inner_bc_dest_pt.i2;
                const int i0src = bcstruct->inner[which_gz][pt].inner_bc_src_pt.i0;
                const int i1src = bcstruct->inner[which_gz][pt].inner_bc_src_pt.i1;
                const int i2src = bcstruct->inner[which_gz][pt].inner_bc_src_pt.i2;
                const int8_t *prty = bcstruct->inner[which_gz][pt].parity;
                gfs[IDX4S(which_gf, i0dest, i1dest, i2dest)] = bcstruct->inner[which_gz][pt].parity[gfs_parity[which_gf]] * gfs[IDX4S(which_gf, i0src, i1src, i2src)];
            }
        }
    }
}


Writing BSSN_LinearScalarFieldEvolution_Ccodes//boundary_conditions/apply_inner_parity_conditions.h


<a id='main'></a>
## Main C Code \[Back to [top](#toc)\]

In [24]:
# Define REAL, set the number of ghost cells NGHOSTS (from NRPy+'s FD_CENTDERIVS_ORDER),
# and set the CFL_FACTOR (which can be overwritten at the command line)

with open(os.path.join(Ccodesdir,"BSSN_Playground_REAL__NGHOSTS__CFL_FACTOR.h"), "w") as file:
    file.write("""
// Part P0.a: Set the number of ghost cells, from NRPy+'s FD_CENTDERIVS_ORDER
#define NGHOSTS """+str(int(FD_order/2)+1)+"""
// Part P0.b: Set the numerical precision (REAL) to double, ensuring all floating point
//            numbers are stored to at least ~16 significant digits
#define REAL """+REAL+"""
// Part P0.c: Set the CFL Factor. Can be overwritten at command line.
REAL CFL_FACTOR = """+str(default_CFL_FACTOR)+";")

In [25]:
%%writefile $Ccodesdir/main.c

// Define REAL and NGHOSTS; and declare CFL_FACTOR. This header is generated in NRPy+.
#include "BSSN_Playground_REAL__NGHOSTS__CFL_FACTOR.h"

#include "rfm_files/rfm_struct__declare.h"

#include "declare_Cparameters_struct.h"

// All SIMD intrinsics used in SIMD-enabled C code loops are defined here:
#include "SIMD/SIMD_intrinsics.h"
#ifdef SIMD_IS_DISABLED
// Algorithm for upwinding, SIMD-disabled version.
// *NOTE*: This upwinding is backwards from
//  usual upwinding algorithms, because the
//  upwinding control vector in BSSN (the shift)
//  acts like a *negative* velocity.
#define UPWIND_ALG(UpwindVecU) UpwindVecU > 0.0 ? 1.0 : 0.0
#endif

// Import needed header files
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <time.h>
#include <string.h>
#include <stdint.h> // Needed for Windows GCC 6.x compatibility
#ifndef M_PI
#define M_PI 3.141592653589793238462643383279502884L
#endif
#ifndef M_SQRT1_2
#define M_SQRT1_2 0.707106781186547524400844362104849039L
#endif
#define wavespeed 1.0 // Set CFL-based "wavespeed" to 1.0.

// Declare the IDX4S(gf,i,j,k) macro, which enables us to store 4-dimensions of
// data in a 1D array. In this case, consecutive values of "i"
// (all other indices held to a fixed value) are consecutive in memory, where
// consecutive values of "j" (fixing all other indices) are separated by
// Nxx_plus_2NGHOSTS0 elements in memory. Similarly, consecutive values of
// "k" are separated by Nxx_plus_2NGHOSTS0*Nxx_plus_2NGHOSTS1 in memory, etc.
#define IDX4S(g, i, j, k) \
    ((i) + Nxx_plus_2NGHOSTS0 * ((j) + Nxx_plus_2NGHOSTS1 * ((k) + Nxx_plus_2NGHOSTS2 * (g))))
#define IDX4ptS(g, idx) ((idx) + (Nxx_plus_2NGHOSTS0 * Nxx_plus_2NGHOSTS1 * Nxx_plus_2NGHOSTS2) * (g))
#define IDX3S(i, j, k) ((i) + Nxx_plus_2NGHOSTS0 * ((j) + Nxx_plus_2NGHOSTS1 * ((k))))
#define LOOP_REGION(i0min, i0max, i1min, i1max, i2min, i2max) \
    for (int i2 = i2min; i2 < i2max; i2++)                    \
        for (int i1 = i1min; i1 < i1max; i1++)                \
            for (int i0 = i0min; i0 < i0max; i0++)
#define LOOP_ALL_GFS_GPS(ii) _Pragma("omp parallel for") for (int(ii) = 0; (ii) < Nxx_plus_2NGHOSTS_tot * NUM_EVOL_GFS; (ii)++)

// Set *GF macros, as well as xxCart()
#include "boundary_conditions/gridfunction_defines.h"

// Set xxCart(const paramstruct *restrict params,
//            REAL *restrict xx[3],
//            const int i0,const int i1,const int i2,
//            REAL xCart[3]),
// which maps xx->Cartesian via
// {xx[0][i0],xx[1][i1],xx[2][i2]}->{xCart[0],xCart[1],xCart[2]}
#include "xxCart.h"

// Defines set_Nxx_dxx_invdx_params__and__xx(const int EigenCoord, const int Nxx[3], paramstruct *restrict params, REAL *restrict xx[3]),
// which sets params Nxx,Nxx_plus_2NGHOSTS,dxx,invdx, and xx[] for
// the chosen Eigen-CoordSystem if EigenCoord==1, or
// CoordSystem if EigenCoord==0.
#include "set_Nxx_dxx_invdx_params__and__xx.h"

// Include basic functions needed to impose curvilinear
// parity and boundary conditions.
#include "boundary_conditions/CurviBC_include_Cfunctions.h"
#include "boundary_conditions/apply_inner_parity_conditions.h"

// Include function for enforcing detgammabar constraint.
#include "enforce_detgammabar_constraint.h"

// Find the CFL-constrained timestep
#include "find_timestep.h"

// Declare function necessary for setting up the initial data.
// Define BSSN_ID() for UIUC Black Hole initial data
// Set the generic driver function for setting up BSSN initial data
#include "initial_data.h"

// Declare and define function for setting up the initial data for the scalar field
#include "fields_initial_data.h"

// Declare function for evaluating Hamiltonian constraint (diagnostic)
#include "Hamiltonian_constraint.h"

// Declare rhs_eval function, which evaluates BSSN RHSs
#include "rhs_eval.h"

// Declare fields_rhs_eval function, which evaluates the scalar field RHSs
#include "fields_rhs_eval.h"

// Declare Ricci_eval function, which evaluates Ricci tensor
#include "Ricci_eval.h"

// Declare function for evaluating real and imaginary parts of psi4 (diagnostic)
#include "coord_defines.h"  // Defines macros related to coordinate systems and psi4 extraction
#ifdef PSI4
#include "psi4.h"
#include "SpinWeight_minus2_SphHarmonics/SpinWeight_minus2_SphHarmonics.h"
#include "psi4_integration.h"
#include "psi4_driver.h"
#endif

void print_all_gfs(const paramstruct *restrict params, const REAL *restrict gfs, const REAL time) {

    #include "set_Cparameters.h"

    char log_filename[] = "checkpoint_data/checkpoint_log.txt";
    FILE *log_test;
    int no_folder = 0;

    if (!(log_test = fopen(log_filename, "w"))) {

        if (time == 0) {
            fprintf(stderr, "WARNING: could not open file %s. Make sure the checkpoint_data directory was created.\n", log_filename);
            fprintf(stderr, "         Checkpoint data will be saved in the current directory.\n");
            fprintf(stderr, "         Move it manually to checkpoint_data, otherwise it won't be read in the next run.\n");
        }

        no_folder = 1;
        sprintf(log_filename, "checkpoint_log.txt");
    }

    FILE *log = fopen(log_filename, "w");
    fprintf(log, "%.5f\n", time);
    fclose(log);

    #pragma omp parallel for
    for (int which_gf = 0; which_gf < NUM_EVOL_GFS; which_gf++) {

        char filename[50];
        if (no_folder == 0) {
            sprintf(filename, "checkpoint_data/gf_%d.txt", which_gf);
        } else {
            sprintf(filename, "gf_%d.txt", which_gf);
        }

        FILE *out = fopen(filename, "w+");

        for (int i2 = 0; i2 < Nxx_plus_2NGHOSTS2; i2++) {
            for (int i1 = 0; i1 < Nxx_plus_2NGHOSTS1; i1++) {
                for (int i0 = 0; i0 < Nxx_plus_2NGHOSTS0; i0++) {
                    fprintf(out, "%e\n", gfs[IDX4S(which_gf, i0, i1, i2)]);
                }
            }
        }
        fclose(out);
    }
}

void initial_data_from_file(const paramstruct *restrict params, REAL * restrict gfs) {

    #include "set_Cparameters.h"

    #pragma omp parallel for
    for (int which_gf = 0; which_gf < NUM_EVOL_GFS; which_gf++) {

        char filename[50];
        sprintf(filename, "checkpoint_data/gf_%d.txt", which_gf);
        FILE *in;
        if (in = fopen(filename, "r+")) {

            for (int i2 = 0; i2 < Nxx_plus_2NGHOSTS2; i2++) {
                for (int i1 = 0; i1 < Nxx_plus_2NGHOSTS1; i1++) {
                    for (int i0 = 0; i0 < Nxx_plus_2NGHOSTS0; i0++) {
                        fscanf(in, "%lf", &gfs[IDX4S(which_gf, i0, i1, i2)]);
                    }
                }
            }

            fclose(in);

        } else {
            fprintf(stderr, "ERROR: could not open file %s.\n", filename);
            fprintf(stderr, "       Make sure the final time of the last run is correct,\n");
            fprintf(stderr, "       and that the checkpoint files are inside the checkpoind_data directory.\n");
            exit(1);
        }
    }

}

REAL get_t_initial() {

    char log_filename[] = "checkpoint_data/checkpoint_log.txt";
    FILE *log_test;

    if (!(log_test = fopen(log_filename, "r+"))) {
        fprintf(stderr, "WARNING: could not find %s, t_initial will be set to 0.\n", log_filename);
        return 0;
    }

    FILE *log = fopen(log_filename, "r+");
    REAL t_initial;
    int n_vals = fscanf(log, "%lf", &t_initial);
    if (n_vals == 0) {
        fprintf(stderr, "WARNING: could not read t_initial from file %s. t_initial set to 0\n", log_filename);
        return 0;
    } else {
        fprintf(stderr, "t_initial is %.5f", t_initial);
    }
    fclose(log);

    return t_initial;

}

int find_extraction_index(const paramstruct *restrict params, REAL *restrict xx[3], const REAL R_EXTRACT) {

    #include "set_Cparameters.h"

    int extraction_idx = NGHOSTS;
    while(extraction_idx < Nxx_plus_2NGHOSTS0) {

        REAL r;

        #ifdef SPHERICAL
        r = xx[0][extraction_idx];
        #endif
        #ifdef SINHSPHERICAL
        r = AMPL * sinh(xx[0][extraction_idx] / SINHW) / sinh(1 / SINHW);
        #endif

        if (r >= R_EXTRACT) break;
        extraction_idx++;
    }

    return extraction_idx;
}

// main() function:
// Step 0: Read command-line input, set up grid structure, allocate memory for gridfunctions, set up coordinates
// Step 1: Set up initial data to an exact solution
// Step 2: Start the timer, for keeping track of how fast the simulation is progressing.
// Step 3: Integrate the initial data forward in time using the chosen RK-like Method of
//         Lines timestepping algorithm, and output periodic simulation diagnostics
// Step 3.a: Output 2D data file periodically, for visualization
// Step 3.b: Step forward one timestep (t -> t+dt) in time using
//           chosen RK-like MoL timestepping algorithm
// Step 3.c: If t=t_final, output conformal factor & Hamiltonian
//           constraint violation to 2D data file
// Step 3.d: Progress indicator printing to stderr
// Step 4: Free all allocated memory
int main(int argc, const char *argv[]) {

    paramstruct params;
#include "set_Cparameters_default.h"

    // Set default initial time to 0
    REAL t_initial = get_t_initial();

    // Read command-line input, error out if nonconformant
    // Expected arguments: Nxx0 Nxx1 Nxx2 [CFL_FACTOR] [chi] [A] [r0] [w] [mu_s]
    // Arguments in brackets are optional

    if (argc < 4 || atoi(argv[1]) < NGHOSTS || atoi(argv[2]) < NGHOSTS || atoi(argv[3]) < 2 /* FIXME; allow for axisymmetric sims */)
    {
        fprintf(stderr, "Error: Expected three command-line arguments: ./exec Nx0 Nx1 Nx2,\n");
        fprintf(stderr, "where Nx[0,1,2] is the number of grid points in the 0, 1, and 2 directions.\n");
        fprintf(stderr, "Nx[] MUST BE larger than NGHOSTS (= %d)\n", NGHOSTS);
        exit(1);
    }
    if (argc == 5)
    {
        CFL_FACTOR = strtod(argv[4], NULL);
        if (CFL_FACTOR > 0.5 && atoi(argv[3]) != 2)
        {
            fprintf(stderr, "WARNING: CFL_FACTOR was set to %e, which is > 0.5.\n", CFL_FACTOR);
            fprintf(stderr, "         This will generally only be stable if the simulation is purely axisymmetric\n");
            fprintf(stderr, "         However, Nx2 was set to %d>2, which implies a non-axisymmetric simulation\n", atoi(argv[3]));
        }
        fprintf(stderr, "WARNING: No initial time or scalar field parameters provided. Defaults will be used instead.\n");
    }
    // if (argc == 6) {
    //     // Check CFL factor
    //     CFL_FACTOR = strtod(argv[4], NULL);
    //     if (CFL_FACTOR > 0.5 && atoi(argv[3]) != 2)
    //     {
    //         fprintf(stderr, "WARNING: CFL_FACTOR was set to %e, which is > 0.5.\n", CFL_FACTOR);
    //         fprintf(stderr, "         This will generally only be stable if the simulation is purely axisymmetric\n");
    //         fprintf(stderr, "         However, Nx2 was set to %d>2, which implies a non-axisymmetric simulation\n", atoi(argv[3]));
    //     }
    //     // Check initial time
    //     t_initial = strtod(argv[5], NULL);
    //     if (t_initial < 0) {
    //         fprintf(stderr, "ERROR: Initial simulation time was chosen to be %f < 0.\n", t_initial);
    //         fprintf(stderr, "       Please select a time that is greater than or equal to 0.\n");
    //         exit(1);
    //     }
    //     fprintf(stderr, "WARNING: No scalar field parameters provided. Defaults will be used instead.\n");
    // }
    if (argc == 10) {
        // Check CFL factor
        CFL_FACTOR = strtod(argv[4], NULL);
        if (CFL_FACTOR > 0.5 && atoi(argv[3]) != 2)
        {
            fprintf(stderr, "WARNING: CFL_FACTOR was set to %e, which is > 0.5.\n", CFL_FACTOR);
            fprintf(stderr, "         This will generally only be stable if the simulation is purely axisymmetric\n");
            fprintf(stderr, "         However, Nx2 was set to %d>2, which implies a non-axisymmetric simulation\n", atoi(argv[3]));
        }
        // Check initial time
        // t_initial = strtod(argv[5], NULL);
        // if (t_initial < 0) {
        //     fprintf(stderr, "ERROR: Initial simulation time was chosen to be %f < 0.\n", t_initial);
        //     fprintf(stderr, "       Please select a time that is greater than or equal to 0.\n");
        //     exit(1);
        // }
        // Check if 0 < chi <= 0.99
        REAL chi = strtod(argv[5], NULL);
        if (chi < 0 || chi > 0.99) {
            fprintf(stderr, "ERROR: Black hole spin is set to %.2f. It must obey 0 < chi <= 0.99.\n", chi);
            exit(1);
        }
        // Check if A is non-zero
        REAL A = strtod(argv[6], NULL);
        if (A == 0) {
            fprintf(stderr, "ERROR: Scalar field amplitude must not be zero.\n");
            exit(1);
        }
        // Check if r0 is non-negative
        REAL r0 = strtod(argv[7], NULL);
        if (r0 < 0) {
            fprintf(stderr, "WARNING: R0 was set to %e, which is < 0. This must be a non-negative number.\n", r0);
            fprintf(stderr, "         The symmetric of the number you provided will be used instead.\n");
            r0 = -r0;
        }
        // Check if w is non-zero
        REAL w = strtod(argv[8], NULL);
        if (w == 0) {
            fprintf(stderr, "ERROR: Scalar field gaussian width must not be zero.\n");
            exit(1);
        }
        // Check if mu_s is non-negative
        REAL mu_s = strtod(argv[9], NULL);
        if (mu_s < 0) {
            fprintf(stderr, "ERROR: mu_s was set to %e, which is < 0. This must be a non-negative number.\n", mu_s);
            exit(1);
        }

        // Set the scalar field parameters
        params.chi = chi;
        params.A = A;
        params.r0 = r0;
        params.w = w;
        params.mu_s = mu_s;
    }
    // If 5 < argc < 9, not enough parameters provided
    if (argc > 6 && argc < 10) {
        fprintf(stderr, "ERROR: not enough parameters provided.\n");
        fprintf(stderr, "       Expected argument format is Nxx0 Nxx1 Nxx2 [CFL_FACTOR] [chi] [A] [r0] [w] [mu_s]\n.");
        exit(1);
    }
    // Set up numerical grid structure, first in space...
    const int Nxx[3] = {atoi(argv[1]), atoi(argv[2]), atoi(argv[3])};
    if (Nxx[0] % 2 != 0 || Nxx[1] % 2 != 0 || Nxx[2] % 2 != 0)
    {
        fprintf(stderr, "Error: Cannot guarantee a proper cell-centered grid if number of grid cells not set to even number.\n");
        fprintf(stderr, "       For example, in case of angular directions, proper symmetry zones will not exist.\n");
        exit(1);
    }

    // Set free parameters, overwriting Cparameters defaults
    //          by hand or with command-line input, as desired.
#include "free_parameters.h"

    // Uniform coordinate grids are stored to *xx[3]
    REAL *xx[3];
    // Set bcstruct
    bc_struct bcstruct;
    {
        int EigenCoord = 1;
        // Call set_Nxx_dxx_invdx_params__and__xx(), which sets
        // params Nxx,Nxx_plus_2NGHOSTS,dxx,invdx, and xx[] for the
        // chosen Eigen-CoordSystem.
        set_Nxx_dxx_invdx_params__and__xx(EigenCoord, Nxx, &params, xx);
        // Set Nxx_plus_2NGHOSTS_tot
#include "set_Cparameters-nopointer.h"
        const int Nxx_plus_2NGHOSTS_tot = Nxx_plus_2NGHOSTS0 * Nxx_plus_2NGHOSTS1 * Nxx_plus_2NGHOSTS2;
        // Find ghostzone mappings; set up bcstruct
#include "boundary_conditions/driver_bcstruct.h"
        // Free allocated space for xx[][] array
        for (int i = 0; i < 3; i++)
            free(xx[i]);
    }

    // Call set_Nxx_dxx_invdx_params__and__xx(), which sets
    // params Nxx,Nxx_plus_2NGHOSTS,dxx,invdx, and xx[] for the
    // chosen (non-Eigen) CoordSystem.
    int EigenCoord = 0;
    set_Nxx_dxx_invdx_params__and__xx(EigenCoord, Nxx, &params, xx);

    // Set all C parameters "blah" for params.blah, including
    // Nxx_plus_2NGHOSTS0 = params.Nxx_plus_2NGHOSTS0, etc.
#include "set_Cparameters-nopointer.h"
    const int Nxx_plus_2NGHOSTS_tot = Nxx_plus_2NGHOSTS0 * Nxx_plus_2NGHOSTS1 * Nxx_plus_2NGHOSTS2;

    // Set timestep based on smallest proper distance between gridpoints and CFL factor
    REAL dt = find_timestep(&params, xx);
    //fprintf(stderr,"# Timestep set to = %e\n",(double)dt);
    int N_final = (int)(t_final / dt + 0.5); // The number of points in time.
                                             // Add 0.5 to account for C rounding down
                                             // typecasts to integers.
    int N_outputs = (int) (t_final - t_initial) * N_outputs_factor;
    int output_every_N = (int)((REAL)N_final / N_outputs);
    if (output_every_N == 0)
        output_every_N = 1;

    // Error out if the number of auxiliary gridfunctions outnumber evolved gridfunctions.
    // This is a limitation of the RK method. You are always welcome to declare & allocate
    // additional gridfunctions by hand.
    if (NUM_AUX_GFS > NUM_EVOL_GFS)
    {
        fprintf(stderr, "Error: NUM_AUX_GFS > NUM_EVOL_GFS. Either reduce the number of auxiliary gridfunctions,\n");
        fprintf(stderr, "       or allocate (malloc) by hand storage for *diagnostic_output_gfs. \n");
        exit(1);
    }

    // Allocate memory for gridfunctions
#include "MoLtimestepping/RK_Allocate_Memory.h"
    REAL *restrict auxevol_gfs = (REAL *)malloc(sizeof(REAL) * NUM_AUXEVOL_GFS * Nxx_plus_2NGHOSTS_tot);

    // Set up precomputed reference metric arrays
    // Allocate space for precomputed reference metric arrays.
#include "rfm_files/rfm_struct__malloc.h"

    // Define precomputed reference metric arrays.
    {
#include "set_Cparameters-nopointer.h"
#include "rfm_files/rfm_struct__define.h"
    }

    if (t_initial == 0) {
        // Set up initial data to an exact solution
        initial_data(&params, xx, y_n_gfs);
        fields_initial_data(&params, xx, y_n_gfs);
    } else {
        // Set up initial data from last run data
        initial_data_from_file(&params, y_n_gfs);
    }

    // Apply inner parity conditions, as initial data
    // are sometimes ill-defined in ghost zones.
    // E.g., spherical initial data might not be
    // properly defined at points where r=-1.
    apply_inner_parity_conditions(&params, &bcstruct, NUM_EVOL_GFS, evol_gf_parity, y_n_gfs);
    enforce_detgammabar_constraint(&rfmstruct, &params, y_n_gfs);

    // Print evolution parameters to command line on the first evolution only
    if (t_initial == 0) {

        FILE *params_file = fopen("run_params.txt", "w");

        fprintf(params_file, "Black hole mass:                 %f\n", params.M);
        fprintf(params_file, "Black hole spin:                 %f\n", params.chi);
        fprintf(params_file, "Scalar field mass:               %f\n", params.mu_s);
        fprintf(params_file, "Scalar field gaussian amplitude: %f\n", params.A);
        fprintf(params_file, "Scalar field gaussian centre:    %f\n", params.r0);
        fprintf(params_file, "Scalar field gaussian width:     %f\n", params.w);
        fprintf(params_file, "Coordinate system:               %s\n", coord_system);
        #ifdef SPHERICAL
        fprintf(params_file, "RMAX =                           %f\n", params.RMAX);
        #endif
        #ifdef SINHSPHERICAL
        fprintf(params_file, "AMPL =                           %f\n", params.AMPL);
        fprintf(params_file, "SINHW =                          %f\n", params.SINHW);
        #endif

        fclose(params_file);
    }

    // Find the psi4 extraction radial index
    #ifdef PSI4
    const int extraction_idx = find_extraction_index(&params, xx, R_EXTRACT);
    #endif

    // Start the timer, for keeping track of how fast the simulation is progressing.
#ifdef __linux__ // Use high-precision timer in Linux.
    struct timespec start, end;
    clock_gettime(CLOCK_REALTIME, &start);
#else // Resort to low-resolution, standards-compliant timer in non-Linux OSs
    // http://www.cplusplus.com/reference/ctime/time/
    time_t start_timer, end_timer;
    time(&start_timer); // Resolution of one second...
#endif

    // Integrate the initial data forward in time using the chosen RK-like Method of
    // Lines timestepping algorithm, and output periodic simulation diagnostics
    for (int n = 0; n <= N_final; n++)
    { // Main loop to progress forward in time.

        // Output 2D data file periodically, for visualization
        if (n % output_every_N == 0)
        {

            REAL current_t = n * dt + t_initial;

            // Compute and print the psi4 decomposition into spherical harmonics
            #ifdef PSI4
            psi4_driver(&params, current_t, extraction_idx, xx, y_n_gfs, diagnostic_output_gfs);
            #endif

            // Evaluate Hamiltonian constraint violation
            Hamiltonian_constraint(&rfmstruct, &params, y_n_gfs, diagnostic_output_gfs);

            char filename[100];
            sprintf(filename, "out_%.5f.txt", current_t);
            FILE *out = fopen(filename, "w");
            LOOP_REGION(NGHOSTS, Nxx_plus_2NGHOSTS0 - NGHOSTS,
                        NGHOSTS, Nxx_plus_2NGHOSTS1 - NGHOSTS,
                        NGHOSTS, Nxx_plus_2NGHOSTS2 - NGHOSTS)
            {
                const int idx = IDX3S(i0, i1, i2);
                REAL xx0 = xx[0][i0];
                REAL xx1 = xx[1][i1];
                REAL xx2 = xx[2][i2];
                fprintf(out, "%e %e %e %e %e %e %e %e\n",
                        xx0, xx1, xx2, y_n_gfs[IDX4ptS(ALPHAGF, idx)], y_n_gfs[IDX4ptS(CFGF, idx)], diagnostic_output_gfs[IDX4ptS(HGF, idx)],
                        y_n_gfs[IDX4ptS(PHIGF, idx)], y_n_gfs[IDX4ptS(PIGF, idx)]);
            }

            fclose(out);
            print_all_gfs(&params, y_n_gfs, current_t);
        }

        // Step forward one timestep (t -> t+dt) in time using
        // chosen RK-like MoL timestepping algorithm
#include "MoLtimestepping/RK_MoL.h"
        // meter_alpha_a_pata(&params, y_n_gfs);

        // If t=t_final, output conformal factor & Hamiltonian
        // constraint violation to 2D data file
        if (n == N_final - 1)
        {

            REAL current_t = n * dt + t_initial;

            // Compute and print the psi4 decomposition into spherical harmonics
            #ifdef PSI4
            psi4_driver(&params, current_t, extraction_idx, xx, y_n_gfs, diagnostic_output_gfs);
            #endif

            // Evaluate Hamiltonian constraint violation
            Hamiltonian_constraint(&rfmstruct, &params, y_n_gfs, diagnostic_output_gfs);

            char filename[100];
            sprintf(filename, "out_%.5f.txt", current_t);
            FILE *out = fopen(filename, "w");
            LOOP_REGION(NGHOSTS, Nxx_plus_2NGHOSTS0 - NGHOSTS,
                        NGHOSTS, Nxx_plus_2NGHOSTS1 - NGHOSTS,
                        NGHOSTS, Nxx_plus_2NGHOSTS2 - NGHOSTS)
            {
                const int idx = IDX3S(i0, i1, i2);
                REAL xx0 = xx[0][i0];
                REAL xx1 = xx[1][i1];
                REAL xx2 = xx[2][i2];
                fprintf(out, "%e %e %e %e %e %e %e %e\n",
                        xx0, xx1, xx2, y_n_gfs[IDX4ptS(ALPHAGF, idx)], y_n_gfs[IDX4ptS(CFGF, idx)], diagnostic_output_gfs[IDX4ptS(HGF, idx)],
                        y_n_gfs[IDX4ptS(PHIGF, idx)], y_n_gfs[IDX4ptS(PIGF, idx)]);
            }

            fclose(out);
            print_all_gfs(&params, y_n_gfs, current_t);
        }

        // Progress indicator printing to stderr

        // Measure average time per iteration
#ifdef __linux__ // Use high-precision timer in Linux.
        clock_gettime(CLOCK_REALTIME, &end);
        const long long unsigned int time_in_ns = 1000000000L * (end.tv_sec - start.tv_sec) + end.tv_nsec - start.tv_nsec;
#else // Resort to low-resolution, standards-compliant timer in non-Linux OSs
        time(&end_timer);                                                 // Resolution of one second...
        REAL time_in_ns = difftime(end_timer, start_timer) * 1.0e9 + 0.5; // Round up to avoid divide-by-zero.
#endif
        const REAL s_per_iteration_avg = ((REAL)time_in_ns / (REAL)n) / 1.0e9;

        const int iterations_remaining = N_final - n;
        const REAL time_remaining_in_mins = s_per_iteration_avg * (REAL)iterations_remaining / 60.0;

        const REAL num_RHS_pt_evals = (REAL)(Nxx[0] * Nxx[1] * Nxx[2]) * 4.0 * (REAL)n; // 4 RHS evals per gridpoint for RK4
        const REAL RHS_pt_evals_per_sec = num_RHS_pt_evals / ((REAL)time_in_ns / 1.0e9);

        // Output simulation progress to stderr
        if (n % 10 == 0)
        {
            fprintf(stderr, "%c[2K", 27);                                                          // Clear the line
            fprintf(stderr, "It: %d t=%.2f dt=%.2e | %.1f%%; ETA %.0f s | t/h %.2f | gp/s %.2e\r", // \r is carriage return, move cursor to the beginning of the line
                    n, t_initial + n * (double)dt, (double)dt, (double)(100.0 * (REAL)n / (REAL)N_final),
                    (double)time_remaining_in_mins * 60, (double)(dt * 3600.0 / s_per_iteration_avg), (double)RHS_pt_evals_per_sec);
            fflush(stderr); // Flush the stderr buffer
        }                   // End progress indicator if(n % 10 == 0)
    }                       // End main loop to progress forward in time.
    fprintf(stderr, "\n");  // Clear the final line of output from progress indicator.

    // Free all allocated memory
#include "rfm_files/rfm_struct__freemem.h"
#include "boundary_conditions/bcstruct_freemem.h"
#include "MoLtimestepping/RK_Free_Memory.h"
    free(auxevol_gfs);
    for (int i = 0; i < 3; i++)
        free(xx[i]);

    return 0;
}


Writing BSSN_LinearScalarFieldEvolution_Ccodes//main.c


<a id='sbatch'></a>
## Make sbatch files for Baltasar \[Back to [top](#toc)\]

In [26]:
import sbatch_maker as sm
import cmdline_helper as cmd
import os

Ccodesdir = os.path.join("BSSN_LinearScalarFieldEvolution_Ccodes/")

if not 'domain_size' in locals():
    domain_size = 1000
    sinh_width = 0.17

# horizon_pts = [15]
# nrs = [30, 45, 60, 75, 90, 105, 120]
# nrs = [x for x in range(120, 280, 20)]
nrs = [1172]
#nrs = [626]
nths = [8]
nphs = [16]

# nth = 8
# nph = 16
cfl = 0.5

M = 1
chis = [0.95]
# chis = [0]
A = 0.003
r0 = 25
w = 10
mu_s = 0.35

mkdirs = False

job_name_base = "nlqb_10"
code_folder = "finals_for_real/nlqb_10"
output_folder_name = 'fields'
nthreads = 48

sbatch_folder = os.path.join(Ccodesdir, 'sbatch')
cmd.mkdir(sbatch_folder)

run_all = open(os.path.join(Ccodesdir, 'run_all'), 'w')

count0 = 0
count = count0
for chi in chis:
    for nr in nrs:
        for nth in nths:
            for nph in nphs:
    # for pts in horizon_pts:
        # nr = sm.find_nr(pts, domain_size, sinh_width, M, chi)
        # print(f"For spin {chi}, nr = {nr} with {pts} points inside the horizon.")
    # nr = 5914
                params = {}
                params['nr'] = nr
                params['nth'] = nth
                params['nph'] = nph
                params['cfl'] = cfl
                params['chi'] = chi
                params['A'] = A
                params['w'] = w
                params['r0'] = r0
                params['mu_s'] = mu_s
                job_name = f"{job_name_base}_{nr}_{nth}_{nph}_{chi}"
                file_path = os.path.join(sbatch_folder, f"job_{count}.sbatch")
                sm.make_sbatch_file(file_path, job_name, code_folder, output_folder_name, params, nthreads, mkdirs)
                count += 1

                run_all.write(f'''mkdir output/fields_{nr}_{nth}_{nph}_{chi}
cd output/fields_{nr}_{nth}_{nph}_{chi}
mkdir checkpoint_data
../exec_local {nr} {nth} {nph} {cfl} {chi} {A} {r0} {w} {mu_s}
cd ../../\n''')

# nrs = [216]
# nths = [x for x in range(4, 24, 4)]
# nphs = [16]
# chis = [0]
#
# for chi in chis:
#     for nr in nrs:
#         for nth in nths:
#             for nph in nphs:
#
#                 params = {}
#                 params['nr'] = nr
#                 params['nth'] = nth
#                 params['nph'] = nph
#                 params['cfl'] = cfl
#                 params['chi'] = chi
#                 params['A'] = A
#                 params['w'] = w
#                 params['r0'] = r0
#                 params['mu_s'] = mu_s
#
#                 job_name = f"{job_name_base}_{nr}_{nth}_{nph}_{chi}"
#
#                 file_path = os.path.join(sbatch_folder, f"job_{count}.sbatch")
#                 sm.make_sbatch_file(file_path, job_name, code_folder, output_folder_name, params, nthreads, mkdirs)
#
#                 count += 1
#
#                 run_all.write(f'''mkdir output/fields_{nr}_{nth}_{nph}_{chi}
# cd output/fields_{nr}_{nth}_{nph}_{chi}
# mkdir checkpoint_data
# ../exec_local {nr} {nth} {nph} {cfl} {chi} {A} {r0} {w} {mu_s}
# cd ../../\n''')
#
# nrs = [216]
# nths = [8]
# nphs = [x for x in range(8, 35, 8)]
# chis = [0]
#
# for chi in chis:
#     for nr in nrs:
#         for nth in nths:
#             for nph in nphs:
#
#                 params = {}
#                 params['nr'] = nr
#                 params['nth'] = nth
#                 params['nph'] = nph
#                 params['cfl'] = cfl
#                 params['chi'] = chi
#                 params['A'] = A
#                 params['w'] = w
#                 params['r0'] = r0
#                 params['mu_s'] = mu_s
#
#                 job_name = f"{job_name_base}_{nr}_{nth}_{nph}_{chi}"
#
#                 file_path = os.path.join(sbatch_folder, f"job_{count}.sbatch")
#                 sm.make_sbatch_file(file_path, job_name, code_folder, output_folder_name, params, nthreads, mkdirs)
#
#                 count += 1
#
#                 run_all.write(f'''mkdir output/fields_{nr}_{nth}_{nph}_{chi}
# cd output/fields_{nr}_{nth}_{nph}_{chi}
# mkdir checkpoint_data
# ../exec_local {nr} {nth} {nph} {cfl} {chi} {A} {r0} {w} {mu_s}
# cd ../../\n''')

run_all.close()

with open(os.path.join(Ccodesdir, 'sbatch_all'), 'w') as file:
    file.write('# /bin/bash\n')
    for i in range(count0, count):
        file.write(f'sbatch sbatch/job_{i}.sbatch\n')